# **IMPORT DATA**

In [39]:
#Import Library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk.corpus
from sklearn.feature_extraction.text import TfidfVectorizer
import codecs
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_auc_score
from sklearn.metrics import f1_score, roc_curve, auc

In [40]:
data = pd.read_excel('Sentimen GOTO.xlsx')
data.head()

,sentiment,tweet
0,Positif,@tanyakanrl barang goto ga ada yg kecewa sih a...
1,Positif,Kinerja Bank Jago (ARTO) Berbalik Positif di 2...
2,Positif,GoTo bentar lagi IPO.. \n\nSiap-siap ya team S...
3,Negatif,"IPO Goto 'Cuma' Rp18 Triliun, Masih Kalah dari..."
4,Positif,IPO GOTO Siapkan Bandar Lokal untuk Jaga Harga...


# **PREPROCESSING DATA**

In [41]:
#Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.pipeline import Pipeline
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import string
import re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [42]:
data = data.dropna()
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 0 to 4949
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  4950 non-null   object
 1   tweet      4950 non-null   object
dtypes: object(2)
memory usage: 77.3+ KB


In [43]:
data['sentiment'] = data['sentiment'].replace(to_replace='Negatif', value='0')
data['sentiment'] = data['sentiment'].replace(to_replace='Positif', value='1')
data.head()

,sentiment,tweet
0,1,@tanyakanrl barang goto ga ada yg kecewa sih a...
1,1,Kinerja Bank Jago (ARTO) Berbalik Positif di 2...
2,1,GoTo bentar lagi IPO.. \n\nSiap-siap ya team S...
3,0,"IPO Goto 'Cuma' Rp18 Triliun, Masih Kalah dari..."
4,1,IPO GOTO Siapkan Bandar Lokal untuk Jaga Harga...


In [44]:
data['sentiment'] = data.sentiment.astype(int)
data.isna().sum() #cek missing value

sentiment    0
tweet        0
dtype: int64

In [45]:
#Case Folding
data['tweet']= data['tweet'].str.lower()
print('Case Folding Result : \n')
print(data['tweet'].head(5))

Case Folding Result : 

0    @tanyakanrl barang goto ga ada yg kecewa sih a...
1    kinerja bank jago (arto) berbalik positif di 2...
2    goto bentar lagi ipo.. \n\nsiap-siap ya team s...
3    ipo goto 'cuma' rp18 triliun, masih kalah dari...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [46]:
#Cleansing
def remove_data(tweet):
# remove tab, new line, ans back slice
  tweet = tweet.replace('\\t',""" """).replace('\\n',""" """).replace('\\u',"""
""").replace('\\',""" """)
# remove non ASCII (emoticon, chinese word, .etc)
  tweet = tweet.encode('ascii', 'replace').decode('ascii')
# remove mention, link, hashtag
  tweet = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",
tweet).split())
# remove incomplete URL
  return tweet.replace("http://", " ").replace("https://", " ")
  

data['tweet'] = data['tweet'].apply(remove_data)
print(data['tweet'].head(5))

0    barang goto ga ada yg kecewa sih aku, jadi pil...
1    kinerja bank jago (arto) berbalik positif di 2...
2    goto bentar lagi ipo.. siap-siap ya team settl...
3    ipo goto 'cuma' rp18 triliun, masih kalah dari...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [47]:
#Remove Number
def remove_number(tweet):
  return re.sub(r"\d+", "", tweet)
data['tweet'] = data['tweet'].apply(remove_number)
print(data['tweet'].head(5))

0    barang goto ga ada yg kecewa sih aku, jadi pil...
1    kinerja bank jago (arto) berbalik positif di ....
2    goto bentar lagi ipo.. siap-siap ya team settl...
3    ipo goto 'cuma' rp triliun, masih kalah dari b...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [48]:
#REMOVE PUNCTUATION
def remove_tanda(tweet):
  return re.sub(r"[^\w\s]","",tweet)
data['tweet'] = data['tweet'].apply(remove_tanda)
def remove_punctuation(tweet):
  return tweet.translate(str.maketrans("","",string.punctuation))
data['tweet'] = data['tweet'].apply(remove_punctuation)
print(data['tweet'].head(5))

0    barang goto ga ada yg kecewa sih aku jadi pili...
1    kinerja bank jago arto berbalik positif di  ha...
2    goto bentar lagi ipo siapsiap ya team settleme...
3    ipo goto cuma rp triliun masih kalah dari buka...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [49]:
#REMOVE WHITESPACE LEADING & TRAILING
def remove_whitespace_LT(tweet):
   return tweet.strip()
data['tweet'] = data['tweet'].apply(remove_whitespace_LT)
print(data['tweet'].head(5))

0    barang goto ga ada yg kecewa sih aku jadi pili...
1    kinerja bank jago arto berbalik positif di  ha...
2    goto bentar lagi ipo siapsiap ya team settleme...
3    ipo goto cuma rp triliun masih kalah dari buka...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [50]:
#REMOVE MULTIPLE WHITESPACE INTO SINGLE SPACE
def remove_whitespace_multiple(tweet):
  return re.sub('\s+',' ',tweet)
data['tweet'] =data['tweet'].apply(remove_whitespace_multiple)
print(data['tweet'].head(5))

0    barang goto ga ada yg kecewa sih aku jadi pili...
1    kinerja bank jago arto berbalik positif di has...
2    goto bentar lagi ipo siapsiap ya team settleme...
3    ipo goto cuma rp triliun masih kalah dari buka...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [51]:
#REMOVE SINGLE CHAR
def remove_singl_char(tweet):
  return re.sub(r"\b[a-zA-Z]\b", "", tweet)
data['tweet'] = data['tweet'].apply(remove_singl_char)
print(data['tweet'].head())

0    barang goto ga ada yg kecewa sih aku jadi pili...
1    kinerja bank jago arto berbalik positif di has...
2    goto bentar lagi ipo siapsiap ya team settleme...
3    ipo goto cuma rp triliun masih kalah dari buka...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [52]:
#Pangkas Huruf Sama Yang Berderet
def repeated_char(text):
    return re.sub("([A-Za-z])\\1{2,}","\\1",text)

data['tweet'] = data['tweet'].apply(repeated_char)
print(data['tweet'].head())

0    barang goto ga ada yg kecewa sih aku jadi pili...
1    kinerja bank jago arto berbalik positif di has...
2    goto bentar lagi ipo siapsiap ya team settleme...
3    ipo goto cuma rp triliun masih kalah dari buka...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [53]:
#Slang Word
def slang_tidak(tweet):
# Transform Slangword
  return tweet.replace(' ga '," tidak ").replace(' gak '," tidak ").replace(' nggak '," tidak ").replace(' gada '," tidak ada ").replace(' gadapet '," tidak dapat ").replace(' dpt '," dapat ").replace(' dptnya '," dapat ").replace(' dapet '," dapat ").replace(' dpet '," dapat ").replace('$buka'," saham bukalapak ").replace(' buka '," bukalapak ").replace('buka lapak'," bukalapak ").replace('$'," ").replace(' cuan '," profit ").replace(' duit '," uang ").replace(' tp '," take profit ").replace(' gede '," besar ").replace(' liat '," lihat ")
    
data['tweet'] = data['tweet'].apply(slang_tidak)
print(data['tweet'].head(5))

0    barang goto tidak ada yg kecewa sih aku jadi p...
1    kinerja bank jago arto berbalik positif di has...
2    goto bentar lagi ipo siapsiap ya team settleme...
3    ipo goto cuma rp triliun masih kalah dari buka...
4    ipo goto siapkan bandar lokal untuk jaga harga...
Name: tweet, dtype: object


In [54]:
#NLT WORD TOKENIZE
import nltk
nltk.download('punkt')
def word_tokenize_wrapper(tweet):
  return word_tokenize(tweet)

data['tweet'] = data['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(data.head(5))
#print('\n\n\n')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenizing Result : 

   sentiment                                              tweet
0          1  [barang, goto, tidak, ada, yg, kecewa, sih, ak...
1          1  [kinerja, bank, jago, arto, berbalik, positif,...
2          1  [goto, bentar, lagi, ipo, siapsiap, ya, team, ...
3          0  [ipo, goto, cuma, rp, triliun, masih, kalah, d...
4          1  [ipo, goto, siapkan, bandar, lokal, untuk, jag...


In [55]:
#IMPORT LIBRARY STOPWORD
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:

# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
# append additional stopword
list_stopwords.extend(["dg", "dgn", "ny", 'klo','kalo', 'amp', 'krn', 'nya', 'nih', 'sih','si', 'tau', 'tuh', 'utk','jd','sdh','aja','nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt','&amp', 'yah', 'banget', 'bgt' , 'jg','yg','hia', 'wk', 'wkw', 'wkwk', 'wkwkw', 'wkwkwk', 'wkwkwkw', 'wkwkwkwk','shm', 'fess', 'xa', 'xb', 'xc','xd', 'xe', 'xf', 'ter', 'ra', 'dar','spy', 'hah', 'sjk', 'rl', 'ijk', 'bg', 'xixixi', 'xixi', 'wae','hia',"py", "xixixixi", "eh", "rb", "br", "mg", "laah", "wkwkwwkwk", "euy", "or", "dll", "xbb", "xaf","esp", "ne", "xapada", "tsb", "et", "xab", "hy", "trs", "tgl", "eits", "kyk", "awowkwo", "nda", "aa","py", "ah", "pny", "dkk", "dlm", "rp", "xae", "wahhhh", "so", "gin", "skr", "gt", "xbe", "ckckckck","xt", "ww", "siih", "st", "xba", "xini", "jdi", "sen", "sblm", "waah", "gtt", "shm", "an", "ama", "lha","kl", "kah", "lae", "xs", "sy", "ewekwekwekwkewek", "awokwowkwok", "the", "byk", "xad", "ez", "xbf","is", "kc", "fess", "stlh", "xef", "mb", "sbg", "ih", "nihh", "aq", "wkkw", "wkwkwkkwwkkw", "tl", "of","ae", "xbc", "yiy", "ihiyy", "laaah", "oyy","msh","waaw","th","kwk","xac","jt", "xdtoo", "yaaaaaaaah", "elah", "weee", "goto" ,'saham', 'gw','trx', 'ipo','udah', 'ywdlah', 'sahamm','sahamsaham',"kwkw", "deh","cuman", "lo", "mbk", "adl", "men", "ntar", "ampe", "thn", "mw", "aplg", "truss", "hrs", "mah", "neh", "xaeipo", "org", "knpa", "meh", "emg", "xixiixi", "ms", "sgt", "wawwwwww", "ngeh", "gu", "wkwkww" ,'luh','wakakak','euuyy','hbs','pts','mch','sptnya','bhkn','kpn','elaah','ieuteh','duh','oh','awokowkkwok','kwkwk','yoook', 'aaaaa','zzzzz','wkakakakak','wkkwkwk','wkwkk','wkwkkwkw','wkwkwkwkw','wkwkwowk','wkwkwwk','wkwkwww','wkwmkwkwkw','wohhh','xaa','xban','cimeeh','ckp','ckup','brrti','bruh','ehh','ehhh','gtgt','hayo','hayolohh','hayook','hayooo','hbs','hc','heheh','hehehe','heheheehe','hehehehehehehe','hhh','hhhhhmmmm','hhhmmmmmm','hmm','hmmm','hohohoh','ieuteh','aw','awokowkkwok','brrti','bruh','bs','bsr','ciiyee','cie','aowkwwkwkwk','siapsiap','hahaaha','hahah','hahaha','hahahaha','hahahahahaha','haiii','aah','ahh','bentar','yup','taun','woh','cuy','dah', 'oiya', 'ya','daftar', 'udh', 'bener','pas','kali','sampe', 'biar', 'kayak', 'gue','to','biar', 'bikin', 'take', 'pake', 'lembar','bikin','pt','tbk','group','gojek','tokopedia', 'triliun', 'miliar', 'juta', 'ribu', 'ratus', 'puluh', 'persen', 'lot'])
list_stopwords.remove('tidak')
list_stopwords.remove('naik')
list_stopwords.remove('dapat')


# convert list to dictionary
list_stopwords = set(list_stopwords)

def remove_stopwords(words):
  return [word for word in words if word not in list_stopwords]
data['tweet'] = data['tweet'].apply(remove_stopwords)

print('Stopwords Result : \n')
print(data['tweet'].head(5))
print('\n\n\n')

Stopwords Result : 

0                       [barang, tidak, kecewa, pilih]
1    [kerja, bank, jago, arto, positif, hasil, kola...
2                                   [team, settlement]
3                             [kalah, bukalapak, buka]
4               [bandar, lokal, jaga, harga, skenario]
Name: tweet, dtype: object






In [71]:

# get stopword indonesia
list_stopwords = stopwords.words('indonesiaAn')
# append additional stopword
list_stopwords.extend(["dg", "dgn", "ny", 'klo','kalo', 'amp', 'krn', 'nya', 'nih', 'sih','si', 'tau', 'tuh', 'utk','jd','sdh','aja','nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt','&amp', 'yah', 'banget', 'bgt' , 'jg','yg','hia', 'wk', 'wkw', 'wkwk', 'wkwkw', 'wkwkwk', 'wkwkwkw', 'wkwkwkwk','shm', 'fess', 'xa', 'xb', 'xc','xd', 'xe', 'xf', 'ter', 'ra', 'dar','spy', 'hah', 'sjk', 'rl', 'ijk', 'bg', 'xixixi', 'xixi', 'wae','hia',"py", "xixixixi", "eh", "rb", "br", "mg", "laah", "wkwkwwkwk", "euy", "or", "dll", "xbb", "xaf","esp", "ne", "xapada", "tsb", "et", "xab", "hy", "trs", "tgl", "eits", "kyk", "awowkwo", "nda", "aa","py", "ah", "pny", "dkk", "dlm", "rp", "xae", "wahhhh", "so", "gin", "skr", "gt", "xbe", "ckckckck","xt", "ww", "siih", "st", "xba", "xini", "jdi", "sen", "sblm", "waah", "gtt", "shm", "an", "ama", "lha","kl", "kah", "lae", "xs", "sy", "ewekwekwekwkewek", "awokwowkwok", "the", "byk", "xad", "ez", "xbf","is", "kc", "fess", "stlh", "xef", "mb", "sbg", "ih", "nihh", "aq", "wkkw", "wkwkwkkwwkkw", "tl", "of","ae", "xbc", "yiy", "ihiyy", "laaah", "oyy","msh","waaw","th","kwk","xac","jt", "xdtoo", "yaaaaaaaah", "elah", "weee", "goto" ,'saham', 'gw','trx', 'ipo','udah', 'ywdlah', 'sahamm','sahamsaham',"kwkw", "deh","cuman", "lo", "mbk", "adl", "men", "ntar", "ampe", "thn", "mw", "aplg", "truss", "hrs", "mah", "neh", "xaeipo", "org", "knpa", "meh", "emg", "xixiixi", "ms", "sgt", "wawwwwww", "ngeh", "gu", "wkwkww" ,'luh','wakakak','euuyy','hbs','pts','mch','sptnya','bhkn','kpn','elaah','ieuteh','duh','oh','awokowkkwok','kwkwk','yoook', 'aaaaa','zzzzz','wkakakakak','wkkwkwk','wkwkk','wkwkkwkw','wkwkwkwkw','wkwkwowk','wkwkwwk','wkwkwww','wkwmkwkwkw','wohhh','xaa','xban','cimeeh','ckp','ckup','brrti','bruh','ehh','ehhh','gtgt','hayo','hayolohh','hayook','hayooo','hbs','hc','heheh','hehehe','heheheehe','hehehehehehehe','hhh','hhhhhmmmm','hhhmmmmmm','hmm','hmmm','hohohoh','ieuteh','aw','awokowkkwok','brrti','bruh','bs','bsr','ciiyee','cie','aowkwwkwkwk','siapsiap','hahaaha','hahah','hahaha','hahahaha','hahahahahaha','haiii','aah','ahh','bentar','yup','taun','woh','cuy','dah', 'oiya', 'ya','daftar', 'udh', 'bener','pas','kali','sampe', 'biar', 'kayak', 'gue','to','biar', 'bikin', 'take', 'pake', 'lembar','bikin','pt','tbk','group','gojek','tokopedia', 'triliun', 'miliar', 'juta', 'ribu', 'ratus', 'puluh', 'persen', 'lot'])
list_stopwords.remove('tidak')
list_stopwords.remove('naik')
list_stopwords.remove('dapat')


# convert list to dictionary
list_stopwords = set(list_stopwords)

def remove_stopwords(words):
  return [word for word in words if word not in list_stopwords]
data['tweet'] = data['tweet'].apply(remove_stopwords)

print('Stopwords Result : \n')
print(data['tweet'].head(5))
print('\n\n\n')

OSError: No such file or directory: 'C:\\Users\\user\\AppData\\Roaming\\nltk_data\\corpora\\stopwords\\indonesiaAn'

In [64]:
# import Sastrawi package
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [65]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemmed
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for document in data['tweet']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
  term_dict[term] = stemmed_wrapper(term)
  print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
  return [term_dict[term] for term in document]

data['tweet'] = data['tweet'].apply(get_stemmed_term)
print(data['tweet'].head())

7882
------------------------
barang : barang
goto : goto
tidak : tidak
ada : ada
yg : yg
kecewa : kecewa
sih : sih
aku : aku
jadi : jadi
pilih : pilih
kinerja : kerja
bank : bank
jago : jago
arto : arto
berbalik : balik
positif : positif
di : di
hasil : hasil
dari : dari
kolaborasi : kolaborasi
bentar : bentar
lagi : lagi
ipo : ipo
siapsiap : siapsiap
ya : ya
team : team
settlement : settlement
aowkwwkwkwk : aowkwwkwkwk
cuma : cuma
rp : rp
triliun : triliun
masih : masih
kalah : kalah
bukalapak : bukalapak
buka : buka
siapkan : siap
bandar : bandar
lokal : lokal
untuk : untuk
jaga : jaga
harga : harga
begini : begini
skenarionya : skenario
dgn : dgn
penawaran : tawar
sama : sama
maka : maka
ini : ini
berbahaya : bahaya
jauh : jauh
lebih : lebih
tinggi : tinggi
hal : hal
bisa : bisa
memicu : picu
kekecewaan : kecewa
investor : investor
pasca : pasca
switching : switching
ke : ke
apakah : apakah
trauma : trauma
akan : akan
berimbas : imbas
pada : pada
target : target
pendanaan : dana
ba

exist : exist
exit : exit
plan : plan
angin : angin
segar : segar
ngertisaham : ngertisaham
orgasme : orgasme
melihat : lihat
semua : semua
tertuju : tuju
ntar : ntar
dijual : jual
serius : serius
pake : pake
nama : nama
bukannya : bukan
sempet : sempet
kontroversi : kontroversi
itu : itu
tech : tech
companies : companies
seluruh : seluruh
dunia : dunia
memang : memang
crash : crash
fed : fed
naikkan : naik
suku : suku
bunga : bunga
perang : perang
russia : russia
timing : timing
agak : agak
berpeluang : peluang
mengumpulkan : kumpul
sebanyakbanyaknya : sebanyakbanyaknya
usaha : usaha
nyata : nyata
anak : anak
muda : muda
nol : nol
mimpi : mimpi
bangun : bangun
tidur : tidur
tajir : tajir
tdk : tdk
khianati : khianat
congratulations : congratulations
ngapain : ngapain
niat : niat
pen : pen
macbook : macbook
tahan : tahan
dlu : dlu
masukin : masukin
aset : aset
kabar : kabar
bang : bang
abalabal : abalabal
apalagi : apalagi
bodong : bodong
lho : lho
menerima : terima
materi : materi
sem

hack : hack
merah : merah
gua : gua
tetep : tetep
hahaha : hahaha
sebelumnya : belum
disclaimer : disclaimer
gampang : gampang
bangetbangetbanget : bangetbangetbanget
analisis : analisis
dengerin : dengerin
penjelasan : jelas
youtube : youtube
uda : uda
ngincer : ngincer
semenjak : semenjak
desasdesusnya : desasdesusnya
pantau : pantau
pergerakan : gera
bisnisnya : bisnis
alih : alih
artos : artos
apps : apps
gencar : gencar
iklannya : iklan
tipi : tipi
setidaknya : tidak
model : model
mengiklankan : iklan
dirinya : diri
kebanyakan : banyak
setartap : setartap
modelnya : model
ngerabaraba : ngerabaraba
iklan : iklan
cakep : cakep
minus : minus
tipikal : tipikal
sekian : sekian
argumen : argumen
harapan : harap
pertama : pertama
doang : doang
lirik : lirik
berada : ada
state : state
invest : invest
kemaren : kemaren
jualin : jualin
beberapa : beberapa
ijo : ijo
pubex : pubex
duluu : duluu
buruan : buru
seems : seems
menjanjikan : janji
iseng : iseng
anget : anget
gapaham : gapaham
mendi

disasah : disasah
habisan : habis
tadi : tadi
makasih : makasih
ka : ka
bantuannya : bantu
sukseskan : sukses
kecipratan : ciprat
mantep : mantep
lesgo : lesgo
shopee : shopee
menggaet : gaet
pelanggan : langgan
hy : hy
mo : mo
mbk : mbk
sis : sis
skr : skr
menghilang : hilang
amerika : amerika
eropa : eropa
hehe : hehe
jawaban : jawab
smg : smg
abang : abang
harap : harap
mengetahuinya : tahu
prinsip : prinsip
uberdll : uberdll
adl : adl
promosi : promosi
diskon : diskon
customer : customer
diakui : aku
industri : industri
relatif : relatif
dimengerti : erti
front : front
runnernya : runnernya
terintegrasi : integrasi
investment : investment
decision : decision
yours : yours
align : align
with : with
profile : profile
kekayaan : kaya
william : william
tanuwijaya : tanuwijaya
pendiri : diri
goodwill : goodwill
lha : lha
boncos : boncos
resiko : resiko
retail : retail
seram : seram
mengintip : intip
percikan : percik
terhitung : hitung
setia : setia
promo : promo
sedikit : sedikit
sedan

being : being
fool : fool
who : who
doesnt : doesnt
consider : consider
past : past
incidents : incidents
bad : bad
pertanyaannya : tanya
matiin : matiin
komentar : komentar
ig : ig
launching : launching
liatliat : liatliat
twitter : twitter
bawa : bawa
sea : sea
ancur : ancur
smkm : smkm
beraniwkwkwk : beraniwkwkwk
come : come
on : on
poin : poin
gih : gih
pengusaha : usaha
online : online
subjudul : subjudul
spekulan : spekulan
beruntung : untung
optimistis : optimistis
membagikan : bagi
alias : alias
kriteria : kriteria
liat : liat
indonesiamari : indonesiamari
dukung : dukung
sebagus : bagus
membuka : buka
peminatan : minat
beres : beres
kesimpulannya : simpul
gausah : gausah
isinya : isi
sehat : sehat
long : long
projection : projection
artinya : arti
rugimya : rugimya
nyebar : nyebar
bermasalah : masalah
solusinya : solusi
kapok : kapok
ambek : ambek
dibeliin : dibeliin
selama : lama
dikendalikan : kendali
perbankan : perban
tumpuan : tumpu
which : which
ngalir : ngalir
nutup : n

...

alhamdulilah : alhamdulilah
panasnya : panas
pengsan : pengsan
supir : supir
art : art
paripurna : paripurna
nangsib : nangsib
ku : ku
sl : sl
kebakaran : bakar
cinta : cinta
expect : expect
less : less
nusakini : nusakini
ombak : ombak
menyedot : sedot
atensi : atensi
mewarnai : warna
menggarap : garap
komperhensif : komperhensif
eksis : eks
berbasis : bas
dicatat : catat
saban : saban
separuh : paruh
berputar : putar
tower : tower
investing : investing
pra : pra
screening : screening
ud : ud
dibandrol : dibandrol
hrg : hrg
rplembar : rplembar
dirasa : rasa
match : match
tonggak : tonggak
bersejarah : sejarah
tentunya : tentu
size : size
katalis : katalis
role : role
hendak : hendak
hore : hore
suksesnya : sukses
canva : canva
paypal : paypal
shopify : shopify
berselancar : selancar
gelombang : gelombang
nggk : nggk
pp : pp
pesenan : sen
kerjasamanya : kerjasamanya
pre : pre
a : a
mintak : mintak
mainin : mainin
slis : slis
yelo : yelo
penundaan : tunda
msk : msk
hnya : hnya
trif : tr

tblada : tblada
infox : infox
permintaannominal : permintaannominal
investoralibabagoogletencenttelkomselsofbank : investoralibabagoogletencenttelkomselsofbank
arbara : arbara
menyongsong : songsong
kelahiran : lahir
aq : aq
kaos : kaos
metal : metal
idul : idul
fitr : fitr
tabuh : tabuh
genderang : genderang
ketertarikan : tari
tercipta : cipta
terwujud : wujud
tonton : tonton
research : research
bbni : bbni
lahir : lahir
lag : lag
ssksk : ssksk
persenan : persen
tuhkan : tuhkan
wkwkwkwkkw : wkwkwkwkkw
gg : gg
acc : acc
dicukupkan : cukup
pagipagi : pagipagi
suaranya : suara
sisain : sisain
kenang : kenang
kenangan : kenang
ngeleg : ngeleg
moso : moso
karo : karo
keangkut : angkut
wkakw : wkakw
melesat : lesat
chuakkz : chuakkz
lepasin : lepasin
weekend : weekend
mencuri : curi
curi : curi
malumalu : malumalu
dihold : dihold
gapunya : gapunya
ngelihat : ngelihat
wkkwwkkwkwkwkwkw : wkkwwkkwkwkwkwkw
wahahaha : wahahaha
stockbitnya : stockbitnya
lemot : lot
manjat : manjat
woy : woy
effe

decacornnya : decacornnya
close : close
bangsat : bangsat
gotoditunggu : gotoditunggu
akar : akar
rumput : rumput
mokasih : mokasih
latah : latah
bocil : bocil
soroti : sorot
listingrn : listingrn
stuju : stuju
bias : bias
chit : chit
salahsatu : salahsatu
wkkwkkwkwl : wkkwkkwkwl
kegedean : gede
scalpingan : scalpingan
sabih : sabih
ngelamar : ngelamar
diperdagangkan : dagang
diq : diq
bangke : bangke
dicarry : dicarry
treasury : treasury
ayangku : ayangku
anjayy : anjayy
peneris : ter
wkwmkwkwkw : wkwmkwkwkw
kodesa : kodesa
diklasifikasikan : klasifikasi
nii : nii
cashing : cashing
megangnya : megangnya
apik : apik
awet : awet
kolaborasinya : kolaborasi
covid : covid
informatika : informatika
komunikasi : komunikasi
terfomo : terfomo
edisi : edisi
antik : antik
kie : kie
poe : poe
ieu : ieu
allout : allout
dijokul : dijokul
offeringipo : offeringipo
busyet : busyet
rontok : rontok
membara : bara
gig : gig
pengganggu : ganggu
numpang : numpang
dongkrak : dongkrak
zeebo : zeebo
agever :

harianya : harianya
dibagibagi : dibagibagi
hariini : hariini
keharga : harga
balikwkwk : balikwkwk
jeff : jeff
bezos : bezos
gandeng : gandeng
seribulembarwkwwk : seribulembarwkwwk
musti : musti
saiki : saiki
delok : dok
diposisi : posisi
wahh : wahh
menumbuhkan : tumbuh
tumbuhan : tumbuh
iindonesi : iindonesi
bangkitkan : bangkit
menawar : tawar
burs : burs
penyedia : sedia
earth : earth
kacauu : kacauu
hengpon : hengpon
stonks : stonks
tumbukan : tumbu
terbebas : bebas
diramal : ramal
wangi : wangi
gopudnya : gopudnya
perusahanperusahaan : perusahanperusahaan
smmt : smmt
belenggu : belenggu
pgen : pgen
bersifat : sifat
akumulatif : akumulatif
terekam : rekam
tabel : tabel
wizards : wizards
dijumlahkan : jumlah
multiple : multiple
voting : voting
warganet : warganet
thankyou : thankyou
voucher : voucher
janjiw : janjiw
stagnan : stagnan
zona : zona
goblok : goblok
jele : jele
ijonya : ijonya
ngefollow : ngefollow
told : told
koran : koran
fluktuasi : fluktuasi
menukik : tukik
patu : 

pertahanan : tahan
digeber : geber
nig : nig
ceritnya : cit
jamet : jamet
kasihan : kasihan
dividend : dividend
harinya : hari
kotak : kotak
tidaklah : tidak
sepatu : sepatu
lapis : lapis
tertembus : tembus
ngobrolngobrol : ngobrolngobrol
terbantu : bantu
dialami : alami
berkaitan : kait
mmang : mmang
diballik : diballik
ego : ego
bisajadi : bisajadi
kelanjutannya : lanjut
gen : gen
terendahnya : rendah
rutin : rutin
instagramnya : instagramnya
nda : nda
mesti : mesti
tangan : tangan
pelanpelan : pelanpelan
diirisiris : diirisiris
turunmu : turun
kutentukan : tentu
ketarik : tarik
uh : uh
grenshoe : grenshoe
habiskah : habis
balapan : balap
menghina : hina
dihina : hina
calm : calm
adakalanya : adakalanya
membuang : buang
sepatunya : sepatu
praclosing : praclosing
pts : pts
wijaya : wijaya
ewekwekwekwkewek : ewekwekwekwkewek
cutlost : cutlost
menipis : tip
kejelasan : jelas
usahanya : usaha
pirtinyiin : pirtinyiin
nyi : nyi
bounce : bounce
semestinya : mesti
sodarasodara : sodarasodara

In [67]:
data.to_excel("goto-clean.xlsx")